In [1]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks
  
print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, \
    NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D


2023-03-24 22:11:50.717048: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 22:11:50.985069: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-24 22:11:52.441873: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/
2023-03-24 22:11:52.442030: W t

<module 'tensorflow._api.v2.version' from '/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>
Num GPUs Available:  1


2023-03-24 22:11:54.349256: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-24 22:11:54.376143: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-24 22:11:54.376187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
batch_size = 16
img_size = [240, 320]

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
df['filename'] = df["image_id"].astype(str) + ".png"

df = df.sample(frac=1).reset_index(drop=True)


print(df)


# new_df = pd.read_csv(r'machine-learning-in-science-ii-2023/new_training_norm.csv')
# new_df['filename'] = new_df["image_id"].astype(str) + "_speed-" + new_df["speed"].astype(str) + "_angle-" + new_df["angle"].astype(str) + ".png"
# #new_df['filename'] = new_df["image_id"].astype(str) + ".png"

#print(new_df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# new_x_train = new_df[0:int(len(new_df) * 0.6)]
# new_x_validate = new_df[int(len(new_df) * 0.6):int(len(new_df) * 0.8)]
# new_x_evaluate = new_df[int(len(new_df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0          7105  0.5000    1.0   7105.png
1         16298  0.5000    0.0  16298.png
2          9422  0.4375    0.0   9422.png
3          4614  0.5625    1.0   4614.png
4          1076  0.6250    1.0   1076.png
...         ...     ...    ...        ...
14821     11628  0.5625    1.0  11628.png
14822       723  0.6250    1.0    723.png
14823      4078  0.6875    1.0   4078.png
14824      6294  0.5000    1.0   6294.png
14825      5939  0.6250    1.0   5939.png

[14826 rows x 4 columns]


In [3]:

shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

#print(train_generator)

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')


Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.


In [4]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

#transfer.trainable = False # freeze the first layers to the imagenet weights

# CNN base
model = models.Sequential()

# model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=shape))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

model.add(transfer)
model.build()
        
model.add(layers.Conv2D(1280, (3, 3), strides= (2,2), activation='relu'))
model.add(layers.BatchNormalization())
#model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

#model.add(layers.Dropout(0.2))
model.add(layers.Flatten()),

# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))

model.add(layers.Dense(32, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))

# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))

2023-03-24 22:11:55.138537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 22:11:55.147410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-24 22:11:55.147470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-24 22:11:55.147509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0

In [5]:
model.build()


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 3, 4, 1280)        14746880  
                                                                 
 batch_normalization (BatchN  (None, 3, 4, 1280)       5120      
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 15360)             0         
                                                                 
 dense (Dense)               (None, 64)                983104    
                                                                 
 batch_normalization_1 (Batc  (None, 64)               2

In [7]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0015,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100


2023-03-24 22:12:02.623928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-24 22:12:11.029394: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-03-24 22:12:18.963688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-24 22:12:19.101340: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f6e979144a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-24 22:12:19.101422: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-03-24 22:12:19.273821: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-24 22:12:20.05

219/556 [==========>...................] - ETA: 1:07 - loss: 0.1407

2023-03-24 22:13:06.963978: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.81GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-24 22:13:06.964036: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.81GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


556/556 [==============================] - 146s 218ms/step - loss: 0.0917 - val_loss: 0.0261
Epoch 2/100
556/556 [==============================] - 125s 225ms/step - loss: 0.0410 - val_loss: 0.0179
Epoch 3/100
556/556 [==============================] - 118s 212ms/step - loss: 0.0273 - val_loss: 0.0162
Epoch 4/100
556/556 [==============================] - 123s 220ms/step - loss: 0.0230 - val_loss: 0.0137
Epoch 5/100
556/556 [==============================] - 122s 220ms/step - loss: 0.0211 - val_loss: 0.0125
Epoch 6/100
556/556 [==============================] - 126s 227ms/step - loss: 0.0197 - val_loss: 0.0138
Epoch 7/100
556/556 [==============================] - 117s 210ms/step - loss: 0.0183 - val_loss: 0.0122
Epoch 8/100
556/556 [==============================] - 132s 237ms/step - loss: 0.0162 - val_loss: 0.0118
Epoch 9/100
556/556 [==============================] - 130s 233ms/step - loss: 0.0165 - val_loss: 0.0113
Epoch 10/100
556/556 [==============================] - 130s 233ms/

In [8]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)


test_datagen = ImageDataGenerator(rescale = 1./255)

test_images = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

print(test_images)

prediction = model.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.
32/32 [==============================] - 5s 106ms/step
(1020, 2)


In [9]:
from sklearn.metrics import classification_report

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)


prediction = model.predict(test_generator, verbose =1)

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('evaluate.csv')



93/93 [==============================] - 10s 103ms/step - loss: 0.0090
Found 2966 validated image filenames.
93/93 [==============================] - 6s 65ms/step


In [10]:
diff = abs(x_evaluate["angle"] - a)

results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":a, "Difference":diff})
results

#results.to_csv('evaluate.csv')

,Train,Predicted,Difference
11860,0.5625,0.631705,0.069205
11861,0.6250,0.429378,0.195622
11862,0.9375,0.777185,0.160315
11863,0.8125,0.746991,0.065509
11864,0.4375,0.479318,0.041818
...,...,...,...
14821,0.5625,0.392814,0.169686
14822,0.6250,0.640609,0.015609
14823,0.6875,0.710362,0.022862
14824,0.5000,0.474212,0.025788


In [11]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":b})
results

#results.to_csv('evaluate.csv')

,Train,Predicted
11860,1.0,1.0
11861,1.0,0.0
11862,1.0,1.0
11863,1.0,1.0
11864,1.0,1.0
...,...,...
14821,1.0,1.0
14822,1.0,1.0
14823,1.0,1.0
14824,1.0,1.0


In [12]:
model.save("model.h5")

In [13]:
def get_ML_model(model_config_path, model_weights_path, _loss_='categorical_crossentropy', _optimizer_='adam', _metrics_='accuracy'):
    # load YAML and create model
    yaml_file = open(model_config_path, 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    loaded_model = model_from_yaml(loaded_model_yaml)

    print("Model loaded")
    # load weights into new model
    loaded_model.load_weights(model_weights_path)
    print("Model_weights_loaded")
    # evaluate loaded model on test data

    loaded_model.compile(
	loss=_loss_, 
	optimizer=_optimizer_, 
	metrics=[_metrics_])    

    print("Model compiled")
    return loaded_model


model_config_file = "E:\\Shelfie\\Shelfie-Algorithm\\ML_Algorithm_Stack\ML_Models\\CNN_Models\\Model_{}_{}\\model_{}_config.yaml".format(retrain_model_ID, model_type, retrain_model_ID)
# sub_model_weights_file = "E:\\Shelfie\\Shelfie-Algorithm\\ML_Algorithm_Stack\ML_Models\\CNN_Models\\Model_{}_CNN_GA\\Model_{}_Checkpoints\\30R29_weights_26-0.0517-0.9803-0.1515-0.9568.hdf5".format(retrain_model_ID, retrain_model_ID)
sub_model_weights_file = "E:\\Shelfie\\Shelfie-Algorithm\\ML_Algorithm_Stack\ML_Models\\CNN_Models\\Model_{}_{}\\Model_{}_Checkpoints\\60_weights_07-0.1071-0.9635-0.1447-0.9559.hdf5".format(retrain_model_ID, model_type, retrain_model_ID)
model = get_ML_model(model_config_file, sub_model_weights_file)
epoch_start = print("Model loaded from epoch {} - {}".format(epoch_start, sub_model_weights_file))

history = model.fit_generator(
    train_generator,
    epochs=n_epochs,
    steps_per_epoch=total_train_samples / BATCH_SIZE,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=total_validation_samples / BATCH_SIZE,
    initial_epoch=epoch_start,
    callbacks=[csv_logger, check_point, early_stopping, tensorboard])






NameError: name 'retrain_model_ID' is not defined

In [ ]:
model.load("model.h5")

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0015,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=100
)